In [12]:
import pandas as pd
import numpy as np

In [101]:
output = pd.read_csv('myCS/output/test2.c.txt',sep='\t',names=['type','line','level','code'])

linesList = output.line.value_counts().index.sort_values()[1:]

### Prepare list

In [122]:
ast = [['TEMPORARY']]
flag_open = False
flag_close = False
for linenum in linesList:
    statement = []
    line = output[output.line==linenum]
    leaf = line[line.type=='LEAF_NODE'].code.values
    
    chk_var_dec = (line.type=='VAR_DECL').tolist()
    if True in chk_var_dec:
        vardec = np.where(chk_var_dec)[0]
        for item in vardec:
            statement = pd.Series(line.code.tolist()[item].split(' '))
            statement = statement[statement!=';']
            statement = statement[statement!='=']
            if True in (line.type=='POINTER').tolist():
                statement = statement.tolist()
                statement.insert(0,'Pointer')
                statement.insert(0,'Decl')
            else:
                statement = statement.tolist()
                statement.insert(0,'Decl')
            ast.append(pd.Series(statement).unique().tolist())
        continue

    if linenum == linesList[0]:
        statement.append('FuncDef')
        statement.append('Decl')
        statement.append('FuncDecl')
        
    if '{' in leaf:
        flag_open = True

    leaf = leaf[leaf!='{']
    leaf = leaf[leaf!=';']
    leaf = leaf[leaf!='(']
    leaf = leaf[leaf!=')']
    leaf = leaf[leaf!=',']

    for item in leaf:
        item = 'Return' if item == 'return' else item
        if item != item:
            continue
        else:
            item
        statement.append(item)
    
    
    if len(statement)!=0:
        
        if True in (line.type=='POINTER').tolist():
            statement.insert(0,'Pointer')
        
        if True in (line.type=='SELECTION').tolist():
            statement.insert(0,'Condition')
            statement.insert(0,'Selection')
            
        if True in (line.type=='ITERATION').tolist():
            statement.insert(0,'ForInit')
            statement.insert(0,'Condition')
            statement.insert(0,'ForExpr')            
            statement.insert(0,'Iteration')
            
        if True in (line.type=='FUNCTION_CALL').tolist():
            statement.insert(0,'ExprList')
            statement.insert(0,'FuncCall')
            if statement == ast[-1]:
                continue
            
            ast.append(statement)

        else:
            if statement == ast[-1]:
                continue
            ast.append(statement)
    
    if flag_open == True:
        ast.append(['Compound'])
        flag_open = False
    
    if '}' in leaf:
        del(ast[-1])
        ast.append(['End'])

del(ast[0])

In [126]:
seq = []
for statement in ast:
    for item in statement:
        seq.append(item)
        
seq = [seq,seq]

In [128]:
from gensim.models.word2vec import Word2Vec
w2v = Word2Vec(seq, size=128, workers=16, sg=1, min_count=3)

In [132]:
w2v.wv.syn0

/data/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


array([[ 3.3567022e-03, -1.0177628e-03,  2.4025124e-03, ...,
         4.0771444e-03,  1.6953956e-03,  7.6778175e-04],
       [-1.4971640e-03, -3.5054961e-03, -2.4734270e-03, ...,
        -7.7362329e-04,  1.5375036e-03, -1.3684791e-03],
       [ 2.4536913e-03, -5.8110006e-04,  2.6108816e-03, ...,
         5.0149712e-04,  3.5400123e-03,  4.1593453e-03],
       ...,
       [-3.1223088e-03,  2.1019471e-03, -5.2197336e-04, ...,
         2.9906996e-03, -7.5932074e-04, -7.1055940e-05],
       [-2.9107344e-03, -4.6533236e-04, -7.9265743e-04, ...,
         3.2657792e-03, -3.2514932e-03, -1.4636759e-03],
       [-5.6690653e-04,  3.0485558e-04, -3.4700339e-03, ...,
         2.4556681e-03,  1.3290419e-03,  2.0696309e-03]], dtype=float32)

In [133]:
blk = pd.read_pickle('../experiment-3-astnn/astnn/data/train/blocks.pkl')

In [134]:
blk

,id,code,label
46887,46887,"[[32, [2, [30, [40, [81]]]]], [6], [2, [9, [4]...",93
4970,4970,"[[32, [2, [30, [40, [4]]]]], [6], [2, [13, [4]...",42
50521,50521,"[[32, [2, [30, [1296, [4]]]]], [6], [11, [36],...",77
38406,38406,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [23...",63
44230,44230,"[[32, [2, [30, [40, [4]]]]], [6], [2, [411, [4...",80
...,...,...,...
14434,14434,"[[32, [2, [30, [75, [2, [13, [4]]]], [74, [4]]...",61
21424,21424,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [9,...",69
2438,2438,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [85...",96
1449,1449,"[[32, [2, [30, [40, [4]]]]], [6], [2, [23, [9,...",104
